In [12]:
import requests
import bs4
import re
import pandas as pd

from bs4 import BeautifulSoup

PREFIX = {
    "서울": "101", "경기": "102", "광주": "103", "대구": "104", "대전": "105",
    "부산": "106", "울산": "107", "인천": "108", "강원": "109", "경남": "110",
    "경북": "111", "전남": "112", "전북": "113", "충북": "114", "충남": "115",
    "제주": "116", "전국": "117", "세종": "118"
}

SUFFIX = {
    "서울": {
        '전체':'000',
        "강남구": "010", "강동구": "020", "강북구": "030", "강서구": "040", "관악구": "050",
        "광진구": "060", "구로구": "070", "금천구": "080", "노원구": "090", "도봉구": "100",
        "동대문구": "110","동작구": "120", "마포구": "130", "서대문구": "140", "서초구": "150",
        "성동구": "160", "성북구": "170", "송파구": "180", "양천구": "190", "영등포구": "200",
        "용산구": "210", "은평구": "220", "종로구": "230","중구": "240","중랑구": "250"
    },
   '경기': {
        '전체':'000',
        '가평군': '010', '고양시': '020', '고양시 덕양구': '030', '고양시 일산동구': '040', '고양시 일산서구': '050',
        '과천시': '060', '광명시': '070', '광주시': '080', '구리시': '090', '군포시': '100',
        '김포시': '110', '남양주시': '120', '동두천시': '130', '부천시': '140', '부천시 소사구': '150',
        '부천시 오정구': '160', '부천시 원미구': '170', '성남시': '180', '성남시 분당구': '190', '성남시 수정구': '200',
        '성남시 중원구': '210', '수원시': '220', '수원시 권선구': '230', '수원시 영통구': '240', '수원시 장안구': '250',
        '수원시 팔달구': '260', '시흥시': '270', '안산시': '280', '안산시 단원구': '290', '안산시 상록구': '300',
        '안성시': '310', '안양시': '320', '안양시 동안구': '330', '안양시 만안구': '340', '양주시': '350',
        '양평군': '360', '여주시': '370', '연천군': '380', '오산시': '390', '용인시': '400',
        '용인시 기흥구': '410', '용인시 수지구': '420', '용인시 처인구': '430', '의왕시': '440', '의정부시': '450',
        '이천시': '460', '파주시': '470', '평택시': '480', '포천시': '490', '하남시': '500',
        '화성시': '510'
    },
     '인천': {
         '전체':'000',
        '강화군': '010', '계양구': '020', '남동구': '030', '동구': '040', '미추홀구': '050',
        '부평구': '060', '서구': '070', '연수구': '080', '옹진군': '090', '중구': '100'
    },'부산': {
        '전체':'000',
        '강서구': '010', '금정구': '020', '기장군': '030', '남구': '040', '동구': '050',
        '동래구': '060', '부산진구': '070', '북구': '080', '사상구': '090', '사하구': '100',
        '서구': '110', '수영구': '120', '연제구': '130', '영도구': '140', '중구': '150',
        '해운대구': '160'
    },
    '대구': {
        '전체':'000',
        '군위군': '090', '남구': '010', '달서구': '020', '달성군': '030', '동구': '040',
        '북구': '050', '서구': '060', '수성구': '070', '중구': '080'
    }, 
    '광주': {
        '전체':'000',
        '광산구': '010', 
        '남구': '020', 
        '동구': '030', 
        '북구': '040', 
        '서구': '050'
    },
    
     '대전': {
         '전체':'000',
        '대덕구': '010', 
        '동구': '020', 
        '서구': '030', 
        '유성구': '040', 
        '중구': '050'
    }, 
     '울산': {  
         '전체':'000',
        '남구': '010',
        '동구': '020',
        '북구': '030',
        '울주군': '040',
        '중구': '050'
    },
     '세종':{
         '세종특별자치시':'000'
     }, 
     '강원': {
         '전체':'000',
        '강릉시': '010', '고성군': '020', '동해시': '030', '삼척시': '040', '속초시': '050',
        '양구군': '060', '양양군': '070', '영월군': '080', '원주시': '090', '인제군': '100',
        '정선군': '110', '철원군': '120', '춘천시': '130', '태백시': '140', '평창군': '150',
        '홍천군': '160', '화천군': '170', '횡성군': '180'
    }, 
     '경남': {
        '전체':'000',
        '거제시': '010', '거창군': '020', '고성군': '030', '김해시': '040', '남해군': '050',
        '밀양시': '060', '사천시': '070', '산청군': '080', '양산시': '090', '의령군': '100',
        '진주시': '110', '창녕군': '120', '창원시': '130', '창원시 마산합포구': '140', '창원시 마산회원구': '150',
        '창원시 성산구': '160', '창원시 의창구': '170', '창원시 진해구': '180', '통영시': '190', '하동군': '200',
        '함안군': '210', '함양군': '220', '합천군': '230'
    },
     '경북': {
         '전체':'000',
        '경산시': '010', '경주시': '020', '고령군': '030', '구미시': '040', '군위군': '050',
        '김천시': '060', '문경시': '070', '봉화군': '080', '상주시': '090', '성주군': '100',
        '안동시': '110', '영덕군': '120', '영양군': '130', '영주시': '140', '영천시': '150',
        '예천군': '160', '울릉군': '170', '울진군': '180', '의성군': '190', '청도군': '200',
        '청송군': '210', '칠곡군': '220', '포항시': '230', '포항시 남구': '240', '포항시 북구': '250'
    },
      '전남': {
          '전체':'000',
        '강진군': '010', '고흥군': '020', '곡성군': '030', '광양시': '040', '구례군': '050',
        '나주시': '060', '담양군': '070', '목포시': '080', '무안군': '090', '보성군': '100',
        '순천시': '110', '신안군': '120', '여수시': '130', '영광군': '140', '영암군': '150',
        '완도군': '160', '장성군': '170', '장흥군': '180', '진도군': '190', '함평군': '200',
        '해남군': '210', '화순군': '220'
    },
    '전북': {
        '전체':'000',
        '고창군': '010', '군산시': '020', '김제시': '030', '남원시': '040', '무주군': '050',
        '부안군': '060', '순창군': '070', '완주군': '080', '익산시': '090', '임실군': '100',
        '장수군': '110', '전주시': '120', '전주시 덕진구': '130', '전주시 완산구': '140',
        '정읍시': '150', '진안군': '160'
    },    
    '충남': {
        '전체':'000',
        '계룡시': '010', '공주시': '020', '금산군': '030', '논산시': '040', '당진시': '050',
        '보령시': '060', '부여군': '070', '서산시': '080', '서천군': '090', '아산시': '100',
        '연기군': '110', '예산군': '120', '천안시': '130', '천안시 동남구': '140', '천안시 서북구': '150',
        '청양군': '160', '태안군': '170', '홍성군': '180'
    },
    '충북': {
        '전체':'000',
        '괴산군': '010', '단양군': '020', '보은군': '030', '영동군': '040', '옥천군': '050',
        '음성군': '060', '제천시': '070', '증평군': '080', '진천군': '090', '청원군': '100',
        '청주시': '110', '청주시 상당구': '120', '청주시 서원구': '130', '청주시 청원구': '140',
        '청주시 흥덕구': '150', '충주시': '160'
    },
    '제주': {
        '전체':'000',
        '서귀포시': '030', '제주시': '040'
    }
}

# 기존 구조: nested 딕셔너리 만들기
req_parameter_nested = {
    region: {
        district: PREFIX[region] + suffix[-3:]
        for district, suffix in districts.items()
    }
    for region, districts in SUFFIX.items()
}

# 평탄화(flatten) + 시단위 전체 지역 코드 추가
req_parameter = {}

for region, districts in req_parameter_nested.items():
    # 시 전체 ('서울') 코드: '강남구'처럼 직접 접근할 수 있도록
    if '전체' in districts:
        req_parameter[region] = districts['전체']  # ex) '서울': '101000'
    # 각 구/군도 추가
    for district, code in districts.items():
        req_parameter[district] = code

req_parameter2={
    '지역별':'domestic',
    '직업별':'job-category',
    '역세권별':'subway',
    'HOT100':'hot100',
    '헤드헌팅':'headhunting'
}

url = "https://www.saramin.co.kr/zf_user/jobs/list/{지역별}?loc_cd={제주}&panel_type=&search_optional_item=n&search_done=y&panel_count=y&preview=y".format(**req_parameter2, **req_parameter)
print(url)
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'
}

res = requests.get(url, headers = req_header)
res.encoding = 'utf-8'

job_link_list = []
logo_list = []
title_list =[]
corp_list = []
local_list = []
exp_list = []
grad_list = []
date_list = []
badge_list = []
if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser') 
    # CSS 선택자
    a_tag_list = soup.select("div.wrap_list_banner ul.list_product li.item a[href*='/zf_user/jobs/relay/view?view_type=list&rec']")
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tag_list:
        # 링크
        job_link_dict = {}
        job_link_dict['링크'] = "https://www.saramin.co.kr" + a_tag['href']
        job_link_list.append(job_link_dict)

        # 로고
        logo_dict = {}
        if len(a_tag.select("span.logo img[src*='banner_logo/company/logo_banner/']")) == 0:
            logo_dict['로고'] = 'Null'
        else:
            img_url = a_tag.select("span.logo img[src*='banner_logo/company/logo_banner/']")
            logo_dict['로고'] = img_url[0]['src']
        logo_list.append(logo_dict)

        # 제목
        title_dict = {}
        if len(a_tag.select("strong.tit")) == 0:
            title_dict['제목'] = 'Null'
        else:
            title = a_tag.select("strong.tit")[0].text
            title_dict['제목'] = title
        title_list.append(title_dict)

        # 회사
        corp_dict = {}
        if len(a_tag.select("span.corp")) == 0:
            corp_dict['회사'] = 'Null'
        else:
            corp = a_tag.select("span.corp")[0].text
            corp_dict['회사'] = corp
        corp_list.append(corp_dict)
        
        # 지역, 경력, 학력
        temp = a_tag.select("ul.desc li")
        local_dict = {}
        exp_dict = {}
        grad_dict = {}
        if len(temp) >= 3:
            local_dict['지역'] = temp[0].get_text(strip=True)
            exp_dict['요구경력'] = temp[1].get_text(strip=True)
            grad_dict['최소학력'] = temp[2].get_text(strip=True)
        else:
            local_dict['지역'] = 'NULL'
            exp_dict['요구경력'] = 'NULL'
            grad_dict['최소학력'] = 'NULL'
        local_list.append(local_dict)
        exp_list.append(exp_dict)
        grad_list.append(grad_dict)

        for region in local_list:
            if region['지역'].endswith(' 외'):
                region['지역'] = region['지역'][:-2]

        # 기간
        date_dict = {}
        if len(a_tag.select("span.date")) == 0:
            date_dict['기간'] = 'Null'
        else:
            date = a_tag.select("span.date")[0].text
            date_dict['기간'] = date
        date_list.append(date_dict)

        clean_date = []
        for item in date_list:
            clean_item = {key: re.sub(r'\s+', ' ', value).strip() for key, value in item.items()}
            clean_date.append(clean_item)

        # 배지
        badge_dict = {}
        if len(a_tag.select("span.badge span")) == 0:
            badge_dict['배지'] = 'Null'
        else:
            badge = a_tag.select("span.badge span")[0].text
            badge_dict['배지'] = badge
        badge_list.append(badge_dict)

    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'에러 코드 1= {res.status_code}')

combined_list = []
for link, logo, title, corp, local, exp, grad, date, badge in zip(job_link_list, logo_list, title_list, corp_list, local_list, exp_list, grad_list, clean_date, badge_list):
    merged_dict = {}
    merged_dict.update(link)
    merged_dict.update(logo)
    merged_dict.update(title)
    merged_dict.update(corp)
    merged_dict.update(local)
    merged_dict.update(exp)
    merged_dict.update(grad)
    merged_dict.update(date)
    merged_dict.update(badge)
    combined_list.append(merged_dict)

# DataFrame으로 변환
df = pd.DataFrame(combined_list)
print(job_link_list)
df.head()

https://www.saramin.co.kr/zf_user/jobs/list/domestic?loc_cd=116000&panel_type=&search_optional_item=n&search_done=y&panel_count=y&preview=y
에러 코드 1= 200
[{'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50518598'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50506754'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50497510'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=46858808'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50457623'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50506754'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50441759'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50430089'}, {'링크': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50441990'}, {'링크': 'h

,링크,로고,제목,회사,지역,요구경력,최소학력,기간,배지
0,https://www.saramin.co.kr/zf_user/jobs/relay/v...,https://www.saraminbanner.co.kr/banner_logo/co...,통합 객실 예약담당 채용,롯데시티호텔,NULL,NULL,NULL,D-7,오늘 뜬
1,https://www.saramin.co.kr/zf_user/jobs/relay/v...,https://www.saraminbanner.co.kr/banner_logo/co...,쿠팡CLS 물류관리자 채용,쿠팡로지스틱스서비스,NULL,NULL,NULL,~04.30(수),연봉상승률 15% 이상
2,https://www.saramin.co.kr/zf_user/jobs/relay/v...,https://www.saraminbanner.co.kr/banner_logo/co...,빌드업연구소 각 부문 채용,(주)겟앤쇼,NULL,NULL,NULL,~05.14(수),마케팅·홍보·조사 스크랩 급상승
3,https://www.saramin.co.kr/zf_user/jobs/relay/v...,https://www.saraminbanner.co.kr/banner_logo/co...,[카지노&호텔] 대규모 공채,(주)엘티엔터테인먼트,NULL,NULL,NULL,~04.30(수),금융·보험 지원 TOP100
4,https://www.saramin.co.kr/zf_user/jobs/relay/v...,https://www.saraminbanner.co.kr/banner_logo/co...,2025년 상반기 신입/경력 모집,(주)한국팜비오,서울 서초구,신입/경력,고졸이상,~04.29(화),오늘 뜬
